# Use case proposed in [issue #51](https://github.com/bsorrentino/langgraph4j/issues/51) by [pakamona](https://github.com/pakamona)

In [1]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "1.0.0-beta1";
String langgraph4jVersion = "1.4-SNAPSHOT";

In [2]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
// %dependency /list-repos
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-langchain4j:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-agent-executor:\{langgraph4jVersion}
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-open-ai:\{langchain4jVersion}

%dependency /resolve

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core:1.3-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-langchain4j:1.3-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-agent-executor:1.3-SNAPSHOT
Adding dependency dev.langchain4j:langchain4j:0.36.2
Adding dependency dev.langchain4j:langchain4j-open-ai:0.36.2
Solving dependencies
Resolved artifacts count: 64
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core/1.3-SNAPSHOT/langgraph4j-core-1.3-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino

**Initialize Logger**

In [3]:
try( var file = new java.io.FileInputStream("./logging.properties")) {
    var lm = java.util.logging.LogManager.getLogManager();
    lm.checkAccess(); 
    lm.readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("AdaptiveRag");


In [4]:
import org.bsc.langgraph4j.state.AgentState;
import java.util.Optional;

public class MyAgentState extends AgentState {

    public MyAgentState(Map<String,Object> initData) {
        super(initData);
    }

    Optional<String> input() {
        return value( "input" );
    }
    Optional<String> orchestratorOutcome() { 
        return value( "orchestrator_outcome" );
    }
}

In [5]:
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.action.NodeAction;

import java.util.Map;
import java.util.ArrayList;
import java.util.concurrent.CompletableFuture;
import java.util.function.Function;

import dev.langchain4j.data.message.ChatMessage;
import dev.langchain4j.data.message.SystemMessage;
import dev.langchain4j.data.message.UserMessage;

import dev.langchain4j.model.chat.ChatLanguageModel;
import dev.langchain4j.model.input.PromptTemplate;
import dev.langchain4j.model.output.Response;

class OrchestratorAgent implements NodeAction<MyAgentState> {

    private final ChatLanguageModel chatLanguageModel;

    public OrchestratorAgent( ChatLanguageModel chatLanguageModel ) {
        this.chatLanguageModel = chatLanguageModel;
    }
 
    public Map<String, Object> apply(MyAgentState state) throws Exception {
        
        var input = state.input().orElseThrow( () -> new IllegalArgumentException("input is not provided!"));

        var userMessageTemplate = PromptTemplate.from("{{input}}").apply(Map.of("input", input));
        
        var messages = new ArrayList<ChatMessage>();
        
        messages.add(new SystemMessage("""
        You are a helpful assistant. Evaluate the user request and if the request concerns a story return 'story_teller' otherwise 'greeting'
        """));
        messages.add(new UserMessage(userMessageTemplate.text()));

        var result = chatLanguageModel.generate( messages );

        return Map.of( "orchestrator_outcome", result.content().text() );
    }

};



In [6]:
import org.bsc.langgraph4j.action.EdgeAction;

class RouteOrchestratorOutcome implements EdgeAction<MyAgentState> {

    public String apply(MyAgentState state) throws Exception {
        
        var orchestrationOutcome = state.orchestratorOutcome()
                                        .orElseThrow( () -> new IllegalArgumentException("orchestration outcome is not provided!"));

        return orchestrationOutcome;
    }

}

In [7]:
class StoryTellerAgent implements NodeAction<MyAgentState> {

    public Map<String, Object> apply(MyAgentState state) throws Exception {
        log.info( "Story Teller Agent invoked");
        return Map.of();
    }
}

In [8]:
class GreetingAgent implements NodeAction<MyAgentState> {

    public Map<String, Object> apply(MyAgentState state) throws Exception {
        log.info( "Greeting Agent invoked");
        return Map.of();
    }
}

In [9]:
import static org.bsc.langgraph4j.action.AsyncNodeAction.node_async;
import static org.bsc.langgraph4j.action.AsyncEdgeAction.edge_async;
import org.bsc.langgraph4j.StateGraph;
import static org.bsc.langgraph4j.StateGraph.START;
import static org.bsc.langgraph4j.StateGraph.END;
import dev.langchain4j.model.openai.OpenAiChatModel;

var model = OpenAiChatModel.builder()
        .apiKey( System.getenv("OPENAI_API_KEY")  )
        .modelName( "gpt-4o-mini" )
        .logResponses(true)
        .maxRetries(2)
        .temperature(0.0)
        .maxTokens(2000)
        .build();

var orchestratorAgent = node_async( new OrchestratorAgent(model) );
var storyTellerAgent = node_async(new StoryTellerAgent());
var greetingAgent = node_async(new GreetingAgent());
var routeOrchestratorOutcome = edge_async( new RouteOrchestratorOutcome() );

var workflow = new StateGraph<>( MyAgentState::new ) 
                .addNode("orchestrator_agent", orchestratorAgent  )
                .addNode("story_teller_agent", storyTellerAgent )
                .addNode("greetings_agent", greetingAgent )
                .addConditionalEdges("orchestrator_agent",
                        routeOrchestratorOutcome,
                        Map.of( "story_teller", "story_teller_agent",
                                "greeting", "greetings_agent" ))
                .addEdge(START, "orchestrator_agent")
                .addEdge("story_teller_agent", END)
                .addEdge("greetings_agent", END);

var app = workflow.compile();        

In [10]:


for( var node : app.stream( Map.of( "input", "tell me a xmas story"))) {
    log.info( "{}", node );
}

START 
NodeOutput{node=__START__, state={input=tell me a xmas story}} 
Story Teller Agent invoked 
NodeOutput{node=orchestrator_agent, state={input=tell me a xmas story, orchestrator_outcome=story_teller}} 
NodeOutput{node=story_teller_agent, state={input=tell me a xmas story, orchestrator_outcome=story_teller}} 
NodeOutput{node=__END__, state={input=tell me a xmas story, orchestrator_outcome=story_teller}} 


In [11]:
for( var node : app.stream( Map.of( "input", "hi there"))) {
    log.info( "{}", node );
}

START 
NodeOutput{node=__START__, state={input=hi there}} 
Greeting Agent invoked 
NodeOutput{node=orchestrator_agent, state={input=hi there, orchestrator_outcome=greeting}} 
NodeOutput{node=greetings_agent, state={input=hi there, orchestrator_outcome=greeting}} 
NodeOutput{node=__END__, state={input=hi there, orchestrator_outcome=greeting}} 
